<a href="https://colab.research.google.com/github/christopherfan/ChrisFanLLMPractice/blob/main/ProductManagerAutoResponseStatement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AutoGenerate Example of "Snarky" Product Manager Responses to Team Requests

Prompt simulates a Product Manager. We gather a list of departments the PM will work with like marketing, sales supports, and we simulate a list of Q/A statement interactions.

---




In [ ]:
!pip install openai -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 16.2 MB/s eta 0:00:00


In [ ]:
!pip install langchain -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.7/520.7 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 3.8 MB/s eta 0:00:00


In [ ]:
## Upload the stakeholders.py file to colab that has the 1) prompt and 2) examples of stakeholder requests types with descriptions used to populate the prompt

https://drive.google.com/file/d/1TiHIHR7Bw8lexJs3zYQdzh-p3inJMw5I/view?usp=sharing

# Import Dependencies

In [ ]:
import  openai, langchain,langchain.prompts, langchain.chat_models, StakeholderRequests,csv, os, pdb


In [ ]:
from langchain import OpenAI, LLMChain
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.chains import ConversationChain
from getpass import getpass
from langchain.chat_models import ChatOpenAI

from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.memory import ConversationBufferMemory, ChatMessageHistory

from StakeholderRequests import stakeholder_request_types, combined_prompt


Manually Set Open API Key. When you run this step, just manualy copy/paste your Open AI Key.

See: https://help.openai.com/en/articles/4936850-where-do-i-find-my-secret-api-key

Your free credits expire after 30 days and you have to add a credit card to use the API.

In [ ]:
openai.api_key = getpass()

··········


Initialize Large Language Model

In [ ]:
chat_llm = ChatOpenAI(
        temperature=0,
        openai_api_key= openai.api_key,
        model_name="gpt-3.5-turbo-0301"
        )

In [ ]:
stakeholder_request_prompt = "You work in a small software start-up as a product manager. As a product manager, you have many different types of stakeholders from different departments such as customer success, technical support, sales, marketing and engineering. They often request many different types of requests for you during the day that take time for you to respond. One such request is a {request_type} which can be defined as  {request_definition} "

## Set Prompt for User Message.

We are loading this prompt from an uploaded file StakeholderRequests.py

These following variables are inserted at the time the call is made


*   {request_type}: The type of the request as defined in the StakeholderRequest block
*   {request_definition}: The definition of the request
*   {example_num}: The number of examples to genereate
*   {dept_name}: Name of the deparment

###Template:
  "You work in a small software start-up as a product manager. As a product manager, you have many different types of stakeholders from different departments such as customer success, technical support, sales, marketing and engineering. They often request many different types of requests for you during the day that take time for you to respond. "

"One such request is a {request_type} which can be defined as {request_definition} "

"Generate {example_num} brief examples from the {dept_name} department impersonating a member of that team. "

"Use first person phrasing. Do not repeat examples. For each example, output the text in a paragraph block and separate each example with a new line. Do not add a prefix such as 'Example 1:' to the each answer. Simply output the answer."

###Filled in Example
"You work in a small software start-up as a product manager. As a product manager, you have many different types of stakeholders from different departments such as customer success, technical support, sales, marketing and engineering. They often request many different types of requests for you during the day that take time for you to respond. One such request is a feature which can be defined as Stakeholders (customers, sales, customer success teams) may request new features or enhancements to existing ones that will improve user experience, address customer pain points, or help close new deals. Generate 2 brief examples from the marketing department impersonating a member of that team. Use first person phrasing. Do not repeat examples. For each example, output the text in a paragraph block and separate each example with a new line. Do not add a prefix such as 'Example 1:' to the each answer. Simply output the answer."










In [ ]:
user_query = "Imagine you are a member of {dept_name} and generate {example_num} brief examples. Use first person phrasing. Do not repeat examples. For each example, output the text in a paragraph block and separate each example with a new line. Do not add a prefix such as 'Example 1:' to the each answer. Simply output the answer."

In [ ]:
combined_prompt = "You work in a small software start-up as a product manager. As a product manager, you have many different types of stakeholders from different departments such as customer success, technical support, sales, marketing and engineering. They often request many different types of requests for you during the day that take time for you to respond. One such request is a {request_type} which can be defined as {request_definition} Generate {example_num} brief examples from the {dept_name} department impersonating a member of that team. Use first person phrasing. Do not repeat examples. For each example, output the text in a paragraph block and separate each example with a new line. Do not add a prefix such as 'Example 1:' to the each answer. Simply output the answer."

## Construct the System and Human Templates using the langchain template Creation

We are using the Langchain library to generate prompt templates. The tutorials are here https://python.langchain.com/en/latest/modules/prompts/prompt_templates/getting_started.html

In [ ]:
human_message_prompt2 = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        template = combined_prompt,
        input_variables= ["dept_name", "example_num", "request_type","request_definition"],
        )
)

new_csv_prompt = HumanMessagePromptTemplate(
    prompt = PromptTemplate(
    template = StakeholderRequests.combined_prompt,
    input_variables = ["dept_name", "example_num", "request_type","request_definition"],
    )
)


In [ ]:
# Debugging code to see the full prompt
example_num = "2"
dept_name = "marketing"
request_type = StakeholderRequests.stakeholder_request_types[0]['request_type']
request_definition = StakeholderRequests.stakeholder_request_types[0]['definition']

print (new_csv_prompt.format(example_num= example_num, dept_name = dept_name, request_type = request_type, request_definition=request_definition))

content="You work in a small software start-up as a product manager. As a product manager, you have many different types of stakeholders from different departments such as customer success, technical support, sales, marketing and engineering. They often request many different types of requests for you during the day that take time for you to respond. One such request is a feature which can be defined as Stakeholders (customers, sales, customer success teams) may request new features or enhancements to existing ones that will improve user experience, address customer pain points, or help close new deals. Generate 2 brief examples from the marketing department impersonating a member of that team. Use first person phrasing. Do not repeat examples. For each example, output the text in a paragraph block and separate each example with a new line. Do not add a prefix such as 'Example 1:' to the each answer. Simply output the answer." additional_kwargs={}


Construct the full chat prompt template combining the System and User Inputs
[link text](https://)

In [ ]:
# construct the full chat Prompt Template
#chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

chat_prompt = ChatPromptTemplate.from_messages([new_csv_prompt])



## Manually generate the examples via the prompt

In [ ]:
# Create a helper function that takes inputs a) llm_array and ouputs to a b)CSV file name

def chatToCSV( llm_array, csv_filename, department, request_type):

# extract newlines and create an array value

  output_array = []
  lines = llm_array.split("\n")

  for line in lines:
    if line != "":
      output_array.append([request_type,department,line])

# Output to .csv file
  isNewFile = False
  csv_file = csv_filename
  if not os.path.exists(csv_file):
    isNewFile = True


  with open(csv_file, "a", newline="") as file:
    writer = csv.writer(file)
    if isNewFile == True:
        writer.writerow(['request_type','department','request'])
    for row in output_array:
      writer.writerow(row)



In [ ]:
# Create the Chain
chain = LLMChain(
    llm=chat_llm,
    prompt=chat_prompt,
    verbose=True,
    )

request_type = StakeholderRequests.stakeholder_request_types[2]['request_type']
request_definition = StakeholderRequests.stakeholder_request_types[2]['definition']

department = "Technical Support"
example_num = "2"

example_value = chain.run(request_type = request_type, request_definition=request_definition,dept_name= department, example_num = example_num)

print (example_value)




> Entering new LLMChain chain...
Prompt after formatting:
Human: You work in a small software start-up as a product manager. As a product manager, you have many different types of stakeholders from different departments such as customer success, technical support, sales, marketing and engineering. They often request many different types of requests for you during the day that take time for you to respond. One such request is a Prioritization Discussions which can be defined as Stakeholders may request meetings to discuss and align on the product roadmap or prioritize specific features, bugs, or improvements based on business goals, customer needs, and resource availability. Generate 2 brief examples from the Technical Support department impersonating a member of that team. Use first person phrasing. Do not repeat examples. For each example, output the text in a paragraph block and separate each example with a new line. Do not add a prefix such as 'Example 1:' to the each answer. Simp

In [ ]:
chatToCSV(example_value, "StakeholderRequests.csv",department, request_type)


In [ ]:
### Loop through the 10 example types

chain = LLMChain(
    llm=chat_llm,
    prompt=chat_prompt,
    verbose=True,
    )

department = "Sales"
example_num = "1"

Stakeholder_Type_Array = StakeholderRequests.stakeholder_request_types

for item in Stakeholder_Type_Array:
  request_type = item['request_type']
  request_definition = item['definition']
  gpt_prompt =  chat_prompt.format(request_type = request_type, request_definition=request_definition,dept_name= department, example_num = example_num)
  #example_value = chain.run(request_type = request_type, request_definition=request_definition,dept_name= department, example_num = example_num)
  print (f"request type {request_type} \n {gpt_prompt}")
  pdb.set_trace()
  #chatToCSV(example_value, "StakeholderRequests.csv",department, request_type)




request type feature 
 Human: You work in a small software start-up as a product manager. As a product manager, you have many different types of stakeholders from different departments such as customer success, technical support, sales, marketing and engineering. They often request many different types of requests for you during the day that take time for you to respond. One such request is a feature which can be defined as Stakeholders (customers, sales, customer success teams) may request new features or enhancements to existing ones that will improve user experience, address customer pain points, or help close new deals. Generate 1 brief examples from the Sales department impersonating a member of that team. Use first person phrasing. Do not repeat examples. For each example, output the text in a paragraph block and separate each example with a new line. Do not add a prefix such as 'Example 1:' to the each answer. Simply output the answer.
> <ipython-input-17-01046bad0cf0>(14)<cell 

## Create Simple ConversationBufferMemory to make sure the AI is not repeating examples.

We want to store the outputs of each GPT response in the "memory" so and instruct the GPT bot to "not make repeats".

Using principles from the Langchain Memory https://python.langchain.com/en/latest/modules/memory/getting_started.html

For the next sections of code, I've basically copy/pasted this tutorial
https://python.langchain.com/en/latest/modules/memory/examples/adding_memory.html

In [ ]:
#memory = ConversationBufferMemory(memory_key = "chat_history")

# Manually Create Simple Chat Message History
history = ChatMessageHistory()


Generate the Chain with the LLM and insert the variable for the department as an input

In [ ]:
# Create the Chain
chain = LLMChain(
    llm=chat_llm,
    prompt=chat_prompt,
    verbose=True,
    )


# Generate an Array of Example Responses to later append to a .csv
example_responses_array = []


## Generate N examples for a specified Department




In [ ]:
# For now I am hard coding a specific department
department = "customer success"


for counter in range(5):
  example_request = chain.predict(dept_name=department, chat_history=history.messages)
  example_responses_array.append(example_request)

for item in example_responses_array:
  print(item)

## Section Just used for Debugging ConversationBufferMemory

Manually run steps to see what are the outputs of the LLM model, what is added to memory and what is re-fed back into the model

In [ ]:
# Actually run the Chat Chain with the input
#print(chain.run("Engineering"))
example_request = chain.predict(dept_name="professional services")

example_responses_array.append(example_request)



> Entering new LLMChain chain...
Prompt after formatting:
System: You work in a small software start-up as a product manager. As a product manager, you have many different types of stakeholders from different departments such as customer success, technical support, sales, marketing and engineering. They often request many different types of requests for you during the day that take time for you to respond. One such request is a ‘feature request’ which can be defined as  ‘Stakeholders (customers, sales, customer success teams) may request new features or enhancements to existing ones that will improve user experience, address customer pain points, or help close new deals.’
Human: For professional services department, generate an example of a feature request impersonating a member of that team. Use first person phrasing. Do not repeat examples. 

> Finished chain.


In [ ]:
print(example_request)


As a member of the professional services department, I would like to request a new feature that allows us to easily track the progress of our projects. It would be great if we could have a dashboard that shows us the status of each project, including the tasks that have been completed, the ones that are in progress, and the ones that are still pending. This would help us to better manage our workload and ensure that we are meeting our deadlines. Additionally, it would be helpful if we could assign tasks to specific team members and set deadlines for each task. This would help us to stay organized and ensure that everyone is on the same page. Thank you for considering my request.


In [ ]:
memory.load_memory_variables({})

{'chat_history': 'Human: professional services\nAI: As a member of the professional services department, I would like to request a new feature that allows us to easily track the progress of our projects. It would be great if we could have a dashboard that shows us the status of each project, including the tasks that have been completed, the ones that are in progress, and the ones that are still pending. This would help us to better manage our workload and ensure that we are meeting our deadlines. Additionally, it would be helpful if we could assign tasks to specific team members and set deadlines for each task. This would help us to stay organized and ensure that everyone is on the same page. Thank you for considering my request.'}

In [ ]:
example_request = chain.predict(dept_name="professional services")


In [ ]:
memory.load_memory_variables({})

In [ ]:
# Actually run the Chat Chain with the input
#print(chain.run("Engineering"))
example_request = chain.predict(dept_name="professional services")

print(example_request)

In [ ]:
memory.load_memory_variables({})

In [ ]:
for item in example_responses_array:
  print(item)

## Section for Debugging simple ChatMessage History

In [ ]:
example_request = chain.run(dept_name="professional services", chat_history=history.messages)
print(example_request)




> Entering new LLMChain chain...
Prompt after formatting:
System: You work in a small software start-up as a product manager. As a product manager, you have many different types of stakeholders from different departments such as customer success, technical support, sales, marketing and engineering. They often request many different types of requests for you during the day that take time for you to respond. One such request is a ‘feature request’ which can be defined as  ‘Stakeholders (customers, sales, customer success teams) may request new features or enhancements to existing ones that will improve user experience, address customer pain points, or help close new deals.’
Human:  Examples: [], For professional services department, generate an example of a feature request impersonating a member of that team. Use first person phrasing. Do not repeat examples.

> Finished chain.
As a member of the professional services department, I would like to request a feature that allows us to easi

In [ ]:
history.add_ai_message(example_request)

In [ ]:
history.messages

[AIMessage(content="As a member of the professional services department, I would like to request a feature that allows us to easily track the progress of our projects and tasks within the software. It would be helpful if we could assign tasks to specific team members, set deadlines, and receive notifications when tasks are completed or overdue. This would help us stay organized and ensure that we are meeting our clients' expectations in a timely manner.", additional_kwargs={})]

In [ ]:
example_request = chain.run(dept_name="professional services", chat_history=history.messages)




> Entering new LLMChain chain...
Prompt after formatting:
System: You work in a small software start-up as a product manager. As a product manager, you have many different types of stakeholders from different departments such as customer success, technical support, sales, marketing and engineering. They often request many different types of requests for you during the day that take time for you to respond. One such request is a ‘feature request’ which can be defined as  ‘Stakeholders (customers, sales, customer success teams) may request new features or enhancements to existing ones that will improve user experience, address customer pain points, or help close new deals.’
Human:  Examples: [AIMessage(content="As a member of the professional services department, I would like to request a feature that allows us to easily track the progress of our projects and tasks within the software. It would be helpful if we could assign tasks to specific team members, set deadlines, and receive not

In [ ]:
print(example_request)

AIMessage(content="As a member of the professional services department, I would like to request a feature that allows us to easily generate reports on project progress and performance. It would be helpful if we could customize the reports to include specific metrics and data points, such as project timelines, budget utilization, and resource allocation. This would help us provide more detailed and accurate updates to our clients and stakeholders, and also help us identify areas for improvement in our project management processes.", additional_kwargs={})


In [ ]:
print(StakeholderRequests.stakeholder_request_types[0])

{'type': 'feature', 'defintion': 'Stakeholders (customers, sales, customer success teams) may request new features or enhancements to existing ones that will improve user experience, address customer pain points, or help close new deals.'}


In [ ]:
StakeholderRequests.stakeholder_request_types[0][defintion]


NameError: ignored